Finance Prespective on Var and CVar
- Variance is always associated to the risk of an asset. it encompasses both the upside and the downside potential of estimated returns but we dont worry too much about the upside but more about the downside. 
- gives us a way to quantify What is the worst outcome could be to a certain confidence interval. 
- if we were to plot a distribution of a porfolio, we can say that we have 95% confidence that the porfolio is above a certain thereshold (likely to be negative). Value at Risk is the maximum loss at that a given period over a certain confidence interval.  
### Asset
- Market-value total asset = $ A_t $
- total liability = $ L $
- Equity = $ E $
- Total Cap = $ T_c $

$$ A_t = L + E $$
$$ E = T_c$$

## Historical Value at Risk

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from pandas_datareader import data as pdr
from scipy.stats import norm, t

In [32]:
end_date = dt.datetime.now()
start_date = end_date - dt.timedelta(days=60)
apl = pdr.get_data_yahoo(["TLS", 'BHP', 'CBA'], start=start_date, end=end_date)
apl.head()

Attributes Adj Close                   Close                    High  \
Symbols          TLS        BHP    CBA   TLS        BHP    CBA   TLS   
Date                                                                   
2022-05-02      7.99  67.400002  0.210  7.99  67.400002  0.210  8.02   
2022-05-03      7.87  67.879997  0.230  7.87  67.879997  0.230  8.07   
2022-05-04      8.39  69.269997  0.250  8.39  69.269997  0.250  8.47   
2022-05-05      7.88  65.790001  0.230  7.88  65.790001  0.230  8.22   
2022-05-06      7.41  65.849998  0.235  7.41  65.849998  0.235  7.78   

Attributes                      Low                    Open                    \
Symbols           BHP    CBA    TLS        BHP    CBA   TLS        BHP    CBA   
Date                                                                            
2022-05-02  67.410004  0.220  7.700  66.000000  0.200  7.75  66.940002  0.220   
2022-05-03  68.430000  0.230  7.780  67.440002  0.170  7.96  67.589996  0.170   
2022-05-04  69.370003  0.250  7.794  66.449997  0.240  7.93  67.150002  0.240   
2022-05-05  68.169998  0.250  7.750  65.099998  0.220  8.22  68.029999  0.250   
2022-05-06  65.970001  0.235  7.300  64.360001  0.235  7.78  65.169998  0.235   

Attributes    Volume                      
Symbols          TLS        BHP      CBA  
Date                                      
2022-05-02  419700.0  3849600.0  31300.0  
2022-05-03  518500.0  2724900.0  62100.0  
2022-05-04  490000.0  4097600.0  15000.0  
2022-05-05  364800.0  4869000.0  56300.0  
2022-05-06  380300.0  3399400.0    500.0

In [2]:
#import data
def get_data(stocks, start, end):
    stock_dt = pdr.get_data_yahoo(stocks, start= start, end=end)
    stock_dt = stock_dt["Close"]
    returns = stock_dt.pct_change()
    meanReturns = returns.mean()
    covMatrix = returns.cov()
    return returns, meanReturns, covMatrix


#portfolio performance
def portfolioPerformance(weights, meanReturns, covMatrix, Time):
    returns = np.sum(meanReturns*weights)*Time
    std = np.sqrt( np.dot(weights.T, np.dot(covMatrix, weights)) ) * np.sqrt(Time)
    return returns, std

In [3]:
stockList = ['CBA', 'BHP', 'TLS', 'NAB', 'WBC', 'STO']
stocks = [stock+".AX" for stock in stockList] #stockprice in AUS
stocks

endDate = dt.datetime.now()
startDate = endDate - dt.timedelta(days=300)
returns, meanReturns, covMatrix = get_data(stocks, start=startDate, end=endDate)

#check
print(returns.head())
print(meanReturns)
print(covMatrix)

Symbols       CBA.AX    BHP.AX    TLS.AX    NAB.AX    WBC.AX    STO.AX
Date                                                                  
2021-09-02       NaN       NaN       NaN       NaN       NaN       NaN
2021-09-03  0.004636  0.009776  0.000000  0.007371 -0.000384  0.006441
2021-09-06  0.003142 -0.003778  0.012987  0.005575 -0.001153 -0.017600
2021-09-07 -0.000196 -0.003555  0.005128 -0.006584  0.001539  0.003257
2021-09-08  0.007637 -0.010228  0.005102  0.006278  0.002305  0.004870
Symbols
CBA.AX   -0.000294
BHP.AX    0.000304
TLS.AX    0.000121
NAB.AX    0.000005
WBC.AX   -0.001192
STO.AX    0.001196
dtype: float64
Symbols    CBA.AX    BHP.AX    TLS.AX    NAB.AX    WBC.AX    STO.AX
Symbols                                                            
CBA.AX   0.000219  0.000048  0.000054  0.000137  0.000127  0.000068
BHP.AX   0.000048  0.000438  0.000051  0.000048  0.000055  0.000190
TLS.AX   0.000054  0.000051  0.000117  0.000048  0.000035  0.000045
NAB.AX   0.000137  0.00004

In [4]:
#portfolio returns
returns = returns.dropna()
weights = np.random.random(len(returns.columns))
weights /= np.sum(weights) #normalise the weightage so that it sums up to one.
returns['portfolio'] = returns.dot(weights)
returns

Symbols,CBA.AX,BHP.AX,TLS.AX,NAB.AX,WBC.AX,STO.AX,portfolio
Date,,,,,,,
2021-09-03,0.004636,0.009776,0.000000,0.007371,-0.000384,0.006441,0.003506
2021-09-06,0.003142,-0.003778,0.012987,0.005575,-0.001153,-0.017600,0.004897
2021-09-07,-0.000196,-0.003555,0.005128,-0.006584,0.001539,0.003257,0.001291
2021-09-08,0.007637,-0.010228,0.005102,0.006278,0.002305,0.004870,0.000822
2021-09-09,-0.023028,-0.016583,-0.015228,-0.015598,-0.018781,-0.025848,-0.017088
...,...,...,...,...,...,...,...
2022-06-23,0.002010,-0.012917,0.002618,0.005187,0.004608,-0.020000,-0.001713
2022-06-24,0.004568,-0.011852,0.015666,-0.004055,-0.007136,-0.017687,0.003007
2022-06-27,0.039707,0.029485,0.010283,0.033679,0.022587,0.022161,0.021411


In [25]:
def historicalVar(returns, alpha=0.05):
    """
    reads in a panda dataframe of returns /a panda series of returns 
    ouput the percentile of the distribution at the given alpha confidence interval
    """
    if isinstance(returns, pd.Series) or isinstance(returns, pd.DataFrame):
        return returns.quantile(alpha)
    
    else:
        raise TypeError("Expected returns to be DataFrame or Series ")
        
def historicalCVar(returns, alpha=0.05):
    """
    reads in a panda dataframe of returns /a panda series of returns 
    ouput the CVar dataframe or series
    """
    if isinstance(returns, pd.Series) or isinstance(returns, pd.DataFrame):
        below = returns <= historicalVar(returns, alpha)
        return returns[below].mean(axis=0)
    
    else:
        raise TypeError("Expected returns to be DataFrame or Series ")

                
print(historicalVar(returns))
print(historicalCVar(returns))

Symbols
CBA.AX      -0.021820
BHP.AX      -0.036297
TLS.AX      -0.014423
NAB.AX      -0.020022
WBC.AX      -0.023402
STO.AX      -0.032672
portfolio   -0.017385
Name: 0.05, dtype: float64
Symbols
CBA.AX      -0.034933
BHP.AX      -0.052414
TLS.AX      -0.022091
NAB.AX      -0.029288
WBC.AX      -0.037835
STO.AX      -0.044326
portfolio   -0.023707
dtype: float64


In [27]:
#lets check portfolio Var
print(historicalVar(returns["portfolio"]))

# 100 days
Time = 100
hVaR = -historicalVar(returns['portfolio'], alpha=0.05)*np.sqrt(Time)
hCVaR = -historicalCVar(returns['portfolio'], alpha=0.05)*np.sqrt(Time)
pRet, pStd = portfolioPerformance(weights, meanReturns, covMatrix, Time)

InitialInvestment = 10000
print('Expected Portfolio Return:      ', round(InitialInvestment*pRet,2))
print('Value at Risk 95th CI    :      ', round(InitialInvestment*hVaR,2))
print('Conditional VaR 95th CI  :      ', round(InitialInvestment*hCVaR,2))

-0.01738477687750525
Expected Portfolio Return:       -28.1
Value at Risk 95th CI    :       1738.48
Conditional VaR 95th CI  :       2370.74


In [30]:
tesla = pdr.get_data_yahoo("TLS", start= startDate, end=endDate, interval="w") #weekly interval
tesla.head(10)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2021-08-30,34.509998,32.500000,32.830002,34.419998,1509000,34.419998
2021-09-06,34.680000,33.060001,34.389999,33.610001,2295500,33.610001
2021-09-13,33.869999,32.230000,33.740002,32.340000,7790100,32.340000
2021-09-20,32.500000,29.450001,31.500000,29.790001,3813200,29.790001
2021-09-27,30.049999,27.750000,29.600000,28.590000,3555600,28.590000
2021-10-04,28.715000,26.400000,28.290001,26.600000,2254100,26.600000
2021-10-11,28.940001,26.510000,26.650000,28.450001,2198600,28.450001
2021-10-18,28.420000,26.570000,28.420000,26.650000,2471700,26.650000
2021-10-25,26.969999,25.410000,26.500000,25.910000,1645900,25.910000


In [31]:
tesla.tail(10)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2022-05-02,8.470,7.300,7.75,7.41,2173300,7.41
2022-05-09,7.850,6.540,7.28,7.54,4676000,7.54
2022-05-16,8.040,7.280,7.44,7.79,2711200,7.79
2022-05-23,10.130,7.710,7.91,10.12,3688600,10.12
2022-05-30,10.265,9.170,10.21,10.04,3120200,10.04
2022-06-06,10.440,8.613,10.30,8.70,2403300,8.70
2022-06-13,8.520,7.385,8.13,8.12,6672300,8.12
2022-06-20,8.770,7.930,8.31,8.28,4211300,8.28
2022-06-27,8.679,8.170,8.49,8.38,1190000,8.38


In [ ]:
def covar_analysis(data, targets, method="stats"):
    """
    data : expects data to be in pandas and series dataframe
    targets : expects a column name (str)
    methods : stats or sklearn 
    """
    if isinstance(data, pd.Series) or isinstance(data, pd.DataFrame):
        covar_matrix = {}
        components = list(data.columns)
        components.remove(target)
        if method == "stats":
            results = [stats_quanreg(X=component, Y=target, data=returns, alpha=0.05) for component in components]
            results = pd.series(results)
        elif method == "sklearn":
            results = [sklearn_quanreg(X=component, Y=target, data=returns, alpha=0.05) for component in components]
            results = pd.series(results)
        else:
            raise TypeError("method can be either stats or sklearn only")
        
        results.index = components
        return results
    
    else:
        raise TypeError("Expected input to be panda dataframe or series")